<a href="https://colab.research.google.com/github/rjm227/Saller/blob/master/TrabalhoFinal_SC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import importlib
if importlib.util.find_spec("slycot") is None:
    !pip install slycot
if importlib.util.find_spec("control") is None:
    !pip install control
if importlib.util.find_spec("scipy") is None:
    !pip install scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

animate = True
def vehicle(v,t,u,load):
    # inputs
    #  v    = velocidade do veiculo (m/s)
    #  t    = tempo (sec)
    #  u    = posição do pedal (-50% to 100%)
    Cd = 0.24    # drag coefficient
    rho = 1.23  # desidade do ar (kg/m^3)
    A = 5.0      #  ́e a  ́area transversal do veıculo (m^2)
    Fp = 30      # parametro de impulso (N/%pedal)
    m = 700      # massa do veiculo (kg)
    # calculo da derivada da velocidade
    dv_dt = (1.0/(m)) * (Fp*u - 0.5*rho*Cd*A*v**2)
    return dv_dt

tf = 70.0                 # tempo final da simulação
nsteps = 71            # time steps
delta_t = tf/(nsteps-1)   # tamanho de cada steps
ts = np.linspace(0,tf,nsteps) # vetor

# simulate step test operation
step = np.zeros(nsteps) # u = valve % open
step[11:] = 25.0       # posição de subida do pedal


# velocidade inicial
v0 = 0.0
# set point
sp = 25.0
# guardar os resultados
vs = np.zeros(nsteps)

load = 0
# simulate with ODEINT
for i in range(nsteps-1):
    u = step[i]
    # clip inputs to -50% to 100%
    if u >= 100.0:
        u = 100.0
    if u <= -50.0:
        u = -50.0
    v = odeint(vehicle,v0,[0,delta_t],args=(u,load))
    v0 = v[-1]   # take the last value
    vs[i+1] = v0 # store the velocity for plotting
    #sps[i+1] = sp
plt.subplot(2,1,1)
plt.plot(ts,vs,'b-',linewidth=3)
plt.plot([0,tf],[25,25],'k--',linewidth=2)
plt.grid('on')
plt.ylabel('Velocidade (m/s)')
plt.legend(['Velocidade','Set Point'],loc=2)
plt.subplot(2,1,2)
plt.plot(ts,step,'r--',linewidth=3)
plt.ylabel('Pedal do acelerador')    
plt.legend(['Pedal do acelerador (%)'])
plt.xlabel('Tempo (seg)')
plt.grid('on')
plt.show()

In [ ]:
def vehicle(v,t,u,load):
    # inputs
    #  v    = vehicle velocity (m/s)
    #  t    = time (sec)
    #  u    = gas pedal position (-50% to 100%)
    #  load = passenger load + cargo (kg)
    Cd = 0.24    # drag coefficient
    rho = 1.23  # air density (kg/m^3)
    A = 5.0      # cross-sectional area (m^2)
    Fp = 30      # thrust parameter (N/%pedal)
    m = 700      # vehicle mass (kg)
    # calculate derivative of the velocity
    dv_dt = (1.0/(m+load)) * (Fp*u - 0.5*rho*Cd*A*v**2)
    return dv_dt

tf = 60.0                 # final time for simulation
nsteps = 61               # number of time steps
delta_t = tf/(nsteps-1)   # how long is each time step?
ts = np.linspace(0,tf,nsteps) # linearly spaced time vector

# simulate step test operation
step = np.zeros(nsteps) # u = valve % open
step[11:] = 50.0       # step up pedal position
# passenger(s) + cargo load
load = 0 # kg
# velocity initial condition
v0 = 0.0
# set point
sp_store = np.zeros(nteps)
sp = 25.0
# for storing the results
vs = np.zeros(nsteps)
sps = np.zeros(nsteps)

ubias = 0.0
Kc = 1.0/1.2
tauI = 20.0
sum_int = 0.0
es = np.zeros(nsteps)
ies = np.zeros(nsteps)

plt.figure(1,figsize=(5,4))
if animate:
    plt.ion()
    plt.show()

# simulate with ODEINT
for i in range(nsteps-1):
    u = step[i]
    if i == 0:
      sp = 0
    if i == 5:
      sp = 25 * 0.5
    if i == 10:
      sp = 25 * 0.8
    if i == 12:
      sp = 0 
    if i == 25:
      sp = 25
    if i == 40:
      sp = 25 * 0.5
    if i == 50:
      sp = 25 * 0.8
    if i == 60:
      sp = 0 
    if i == 70:
      sp = 0 
    sp_store[i + 1] = sp
    error = sp - V0
    es[i +1] = error
    sum_int = sum_int + error * delta_t
    u = ubias + Kc * error + Kc/tauI * sum_int

    # pedal de  entrada para -50% a 100%
    if u >= 100.0:
        u = 100.0
        sum_int = sum_int - error * delta_t
    if u <= -50.0:
        u = -50.0
        sum_int = sum_int - error * delta_t
    ies[i+1] = sum_int
    step[i+1] = u
    v = odeint(vehicle,v0,[0,delta_t],args=(u,load))
    v0 = v[-1]   # take the last value
    vs[i+1] = v0 # store the velocity for plotting
    

# plot results

plt.figure()
plt.subplot(2,2,1)
plt.plot(ts,vs,'b-',linewidth=3)
plt.plot(ts,sp_store,'k--',linewidth=2)
plt.ylabel('Velocidade (m/s)')
plt.legend(['Velocidade','Set Point'],loc='best')
plt.subplot(2,2,2)
plt.plot(ts,step,'r--',linewidth=3)
plt.ylabel('Pedal do acelerador')    
plt.legend(['Pedal do acelerador (%)',loc = 'best'])
plt.subplot(2,2,3)
plt.plot(ts,es,'b-',linewidth=3)
plt.xlabel(['Erro (SP-PV)'])    
plt.legend(['Pedal do acelerador (%)',loc = 'best'])
    if animate:
        plt.clf()
        plt.subplot(2,1,1)
        plt.plot(ts[0:i+1],vs[0:i+1],'b-',linewidth=3)
        plt.plot(ts[0:i+1],sps[0:i+1],'k--',linewidth=2)
        plt.ylabel('Velocity (m/s)')
        plt.legend(['Velocity','Set Point'],loc='best')
        plt.subplot(2,1,2)
        plt.plot(ts[0:i+1],step[0:i+1],'r--',linewidth=3)
        plt.ylabel('Gas Pedal')    
        plt.legend(['Gas Pedal (%)'])
        plt.xlabel('Time (sec)')
        plt.pause(0.1)    

if not animate:
    # plot results
    plt.subplot(2,1,1)
    plt.plot(ts,vs,'b-',linewidth=3)
    plt.plot(ts,sps,'k--',linewidth=2)
    plt.ylabel('Velocity (m/s)')
    plt.legend(['Velocity','Set Point'],loc=2)
    plt.subplot(2,1,2)
    plt.plot(ts,step,'r--',linewidth=3)
    plt.ylabel('Gas Pedal')    
    plt.legend(['Gas Pedal (%)'])
    plt.xlabel('Time (sec)')
    plt.show()